In [2]:
# import packages

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [3]:
url = 'https://en.wikipedia.org/wiki/Tesla,_Inc.'

downloaded_html = requests.get(url)
soup = BeautifulSoup(downloaded_html.text)

with open('downloaded.html', 'w') as f:
    f.write(soup.prettify())

In [4]:
# select table.wikitable

full_table = soup.select('table.wikitable tbody')[1]

In [5]:
# extract the table column headings -> a list with all column headings

table_head = full_table.select('tr th')

table_columns = []
for element in table_head:
    column_label = element.get_text(separator=' ', strip=True).replace(' ', '_')
    table_columns.append(column_label)

In [6]:
# extract the table data (rows) -> multidimensional list with a list for each row

table_rows = full_table.select('tr')
regex = r'\[\d{1,4}\]'

table_data = []
for index, element in enumerate(table_rows):
    if index > 0:
        row_list = []
        values = element.select('td')
        for value in values:
            value = re.sub(regex, '', value.text.strip()).replace(',', '')
            row_list.append(value)
        table_data.append(row_list)

In [9]:
df = pd.DataFrame(table_data, columns=table_columns)

,Year,Revenue_(mil._USD),Net_income_(mil._USD),Total_assets_(mil._USD),Employees
0,2005,0,−12,8,
1,2006,0,−30,44,70
2,2007,0.073,−78,34,268
3,2008,15,−83,52,252
4,2009,112,−56,130,514
5,2010,117,−154,386,899
6,2011,204,−254,713,1417
7,2012,413,−396,1114,2914
8,2013,2013,−74,2417,5859
9,2014,3198,−294,5831,10161


In [8]:
df.to_csv('tesla.csv', index=False, header=table_columns)